In [6]:
from aoptk.europepmc import EuropePMC
from tempfile import tempdir

from aoptk.scispacy_find_chemical import ScispacyFindChemical
import itertools

import functools
import operator

In [7]:
from aoptk.provide_mesh_term_dataframe_from_xml import ProvideMeshTermDataframeFromXML
from aoptk.mesh_terms import MeshTerms

In [8]:
query = "hepg2 thioacetamide"
epmc = EuropePMC(query, storage = '/tmp/pdfs')

In [9]:
# pdfs = epmc.pdfs()

In [10]:
abstracts = epmc.get_abstracts()

In [11]:
import re


class RegexSentenceGenerator():
    """Generate sentences using regular expressions."""

    def generate_sentences(self, text: str) -> list[str]:
        """Use regex to generate sentences."""
        sentences = re.split(r"(?<=[.!?])\s+|(?<=[.!?])(?=[A-Z])|<h4>|</h4>", str(text))
        return [str(sentence.strip()) for sentence in sentences if sentence.strip()]

In [12]:
model = ScispacyFindChemical()
sg = RegexSentenceGenerator()

/home/hechth/micromamba/envs/aoptk-dev/lib/python3.12/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [21]:
from aoptk.html_tag_remover import HTMLTagRemover
tag_remover = HTMLTagRemover()

In [22]:
sentences_per_pub = list(map(lambda x: sg.generate_sentences(tag_remover.clean(x.text)), abstracts))

In [23]:
chemicals_per_pub = [[model.find_chemical(sentence) for sentence in sentences] for sentences in sentences_per_pub[:10]]

In [24]:
flattened = list(map(lambda x: functools.reduce(operator.iconcat, x, []), chemicals_per_pub))

In [25]:
non_empty = list(filter(lambda x: len(x) > 0, flattened))

In [ ]:
print([[chem.name for chem in pub] for pub in non_empty])

[['folic acid-induced'], ['cp', 'cp', 'cck-8', 'cp', 'cp', 'cp', 'cp', 'mda', 'ros', 'cp', 'cp', 'cp', 'cp'], ['fatty acid', 'alanine', 'aspartate', 'cholesterol', 'tc', 'triglyceride'], ['rhamnetin 3-o-α-rhamnoside', 'thioacetamide'], ['glucose', 'canagliflozin', 'mg/kg diethyl nitrosamine', 'thioacetamide', 'canagliflozin', 'canagliflozin reduced renal oxidative stress', 'canagliflozin', 'canagliflozin'], ['curcumin', 'curcumin', 'curcumin', 'curcumin'], ['non-smad', 'pirfenidone', 'hydronidone'], ['epigallocatechin-3-gallate', 'egcg', 'green tea', 'egcg', 'egcg', 'egcg', 'egcg', 'egcg', 'egcg']]


In [18]:
normalizer = MeshTerms(ProvideMeshTermDataframeFromXML('/home/hechth/dev/aoptk/desc2025.xml').provide_normalization_dataframe())

In [27]:
normalized = [[normalizer.normalize_chemical(chem) for chem in pub] for pub in non_empty]

In [28]:
print([[chem.name for chem in pub] for pub in normalized])

[['folic acid-induced'], ['cp', 'cp', 'sincalide', 'cp', 'cp', 'cp', 'cp', 'mda', 'ros', 'cp', 'cp', 'cp', 'cp'], ['fatty acids', 'alanine', 'aspartic acid', 'cholesterol', 'tc', 'triglycerides'], ['rhamnetin 3-o-α-rhamnoside', 'thioacetamide'], ['glucose', 'canagliflozin', 'mg/kg diethyl nitrosamine', 'thioacetamide', 'canagliflozin', 'canagliflozin reduced renal oxidative stress', 'canagliflozin', 'canagliflozin'], ['curcumin', 'curcumin', 'curcumin', 'curcumin'], ['non-smad', 'pirfenidone', 'hydronidone'], ['epigallocatechin-3-gallate', 'egcg', 'tea', 'egcg', 'egcg', 'egcg', 'egcg', 'egcg', 'egcg']]


In [ ]:
class Processor:
    def __init__(self, model, sentence_generator, num_workers, normalizer = None, cleaner = None):
        self.model = model
        self.normalizer = normalizer
        self.sentence_generator = sentence_generator
        self.cleaner = cleaner
    
    def find_chemicals(self, pubs: list[Abstracts]):
        sentences_per_pub = list(map(lambda x: self.sentence_generator.generate_sentences(self.cleaner.clean(x.text)), pubs))

    def find_associations():
        pass